# Augmenting Xray micro-CT data with MICP data for high resolution pore-microstructural and flow modelling of carbonate rocks.
### Extracting pore size distribution from xray data.

Lead Invsestigator: Olubukola Ishola (olubukola.ishola@okstate.edu)\
Co-Investigator: Javier Vilcaez\
Paper:\
Code base:

### Importing Packages

In [ ]:
import numpy as np
import porespy as ps
import imageio as io
import os
import pandas as pd

### Adjust Settings?

In [ ]:
ps.visualization.set_mpl_style()
np.random.seed(1)

### Helper Functions

In [ ]:
def xray_to_psd(filename,
                resolution,
                sub_volume=[100],
                file_path=False,
                save_path=False,
                return_psd=False,
                save_psd=True,
                save_rev=True):
    """
    Converts binary segmented xray images of rocks to pore size distribution. Pore is White=255, background is black=0. Note that size here means radius.

    Parameters
    ----------
    filename : str
        Name of file including extension.
    resolution : float
        Resolution of image in microns.
    subvolume : float, optional
        Percentage of length scale to process. Must be between 0 and 100. Defaults is 100.
    file_path : str, optional
        Directory of folder where input file is located. Defaults is working directory.
    save_path : str, optional
        Directory of folder where output file is saved. Defaults is working directory.
    return_psd : bolean, optional
        If you want the pore size distribution returned. Default is False..
    save_psd : str, optional
        If you want the pore size distribution saved. Default is True.
    save_rev : str, optional
        If you want to return data on porosity against volume for REV analyis. Default is True.

    Returns
    -------
    Pore size distribution : dataframe, optional

    """

    if file_path:
        file = os.path.join(file_path, filename)
    else:
        file = filename

    image = io.volread(file)
    image = np.invert(image)
    image //= 255
    len_ = min(image.shape)
    image_volume = image.shape[0] * image.shape[1] * image.shape[2]
    sample_porosity = np.sum(image) / image_volume

    volume_rev = []
    volume_rev.append(image_volume * ((resolution / 1000)**3))
    porosity_rev = []
    porosity_rev.append(sample_porosity)
    print(
        f"Porosity of full image is {round(sample_porosity*100, 2)}% and the corresponding volume is {round(volume_rev[0], 2)} mm3."
    )

    for sub in sub_volume:
        sub /= 100
        sub *= len_
        sub = int(sub)
        sub_image = image[:sub, :sub, :sub]

        snow = ps.filters.snow_partitioning(im=sub_image)
        regions = snow.regions * snow.im
        props = ps.metrics.regionprops_3D(regions)
        df = ps.metrics.props_to_DataFrame(props)['volume']

        sub_porosity = df.sum() / (sub**3)
        sub_image_volume = (sub * (resolution / 1000))**3
        volume_rev.append(sub_image_volume)
        porosity_rev.append(sub_porosity)
        print(
            f"Porosity of subvolume is {round(sub_porosity*100, 2)}% and the corresponding subvolume is {round(sub_image_volume, 2)} mm3."
        )

        excel_name = str(
            sub_image_volume)[:5] + '_mm3_' + filename[:-4] + '.xlsx'

        if save_path:
            excel_name = os.path.join(save_path, excel_name)

        df = pixel_volume_to_pore_radius(df, resolution=resolution)
        df = df.rename(columns={'volume': 'Pore radius (microns)'})

        if save_psd:
            df.to_excel(excel_name, index=False)

        if return_psd:
            return df

    if save_rev:
        rev_df = pd.DataFrame({
            'volume (mm3)': volume_rev,
            'porosity (%)': porosity_rev,
        })

        rev_name = filename[:-4] + '.xlsx'

        if save_path:
            rev_name = os.path.join(save_path, rev_name)

        rev_df.to_excel(rev_name, index=False)

In [ ]:
def pixel_volume_to_pore_radius(pixel_volume, resolution):
    """
    Converts pixel volume to the radius of an equivalent sphere.      

    Parameters
    ----------
    pixel_volume : float
        Volume reported by porespy in pixels.
    resolution : float
        Resolution of image in microns.

    Returns
    -------
    Pore radius : float
        pore radiusof equivalent sphere.
    
    """

    pixel_pore_radius = ((3 / 4) * (1 / 3.14) * pixel_volume)**(1 / 3)
    pore_radius = pixel_pore_radius * resolution

    return pore_radius

### Inputs

In [ ]:
filename = 'SampleA.tif'
file_path = r'C:\Users\oishola\OneDrive - Oklahoma A and M System\Doctoral Dissertation\Projects\micp_xray_for_permeability_prediction_paper\segmented_microct_images'
resolution = 7.5
sub_volume = [10, 20]

### Run Program

In [ ]:
xray_to_psd(filename, resolution, sub_volume, file_path=file_path)